In [1]:
import pandas as pd
import numpy as np

In [2]:
from catboost import cv

In [3]:
from catboost import CatBoostRegressor

In [4]:
from catboost import Pool

In [5]:
loc_lbl = r"D:\TP2_Machine_Learning\Labels"

In [6]:
loc_ftr = r"D:\TP2_Machine_Learning\Features\ftr_auc"

In [7]:
auc_lb = pd.read_csv( loc_lbl + "\\auc_lb.csv", dtype = {"obs_18_20":"bool", "obs_21_23":"bool", "obs_24_26":"bool"} )

### Filtro de manera que tenga los datos necesarios

In [8]:
rh_18_20 = pd.read_csv( r"D:\TP2_Machine_Learning\Training Sets\auc_18_20.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_18_20 = rh_18_20["ref_hash"].drop_duplicates().tolist()
rh_21_23 = pd.read_csv( r"D:\TP2_Machine_Learning\Training Sets\auc_21_23.csv" ).rename( columns = {"device_id":"ref_hash"} ); rh_21_23 = rh_21_23["ref_hash"].drop_duplicates().tolist()

In [9]:
dt_18_20 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_18_20 ), ["ref_hash"] ] #Datos del (18-20)
st_21_23 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_18_20 ), ["21_23_st"] ] #SC del (21-23)

dt_21_23 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_21_23 ), ["ref_hash"] ] #Datos del (21-23)
st_24_26 = auc_lb.loc[ auc_lb["ref_hash"].isin( rh_21_23 ), ["24_26_st"] ] #SC del (24-26)

# <span style="color:orange">  Preparo los datos para predecir </span>
Con los datos 21-23 predecimos "24-26_sc"

In [10]:
Z = dt_21_23

### Agregamos los features 

In [11]:
ftr_01_Z = pd.read_csv( loc_ftr + "\\hora_auc_21_23.csv" ) 
ftr_02_Z = pd.read_csv( loc_ftr + "\\cant_clk_21_23.csv" ) 
ftr_03_Z = pd.read_csv( loc_ftr + "\\cant_ins_21_23.csv" ) 
ftr_04_Z = pd.read_csv( loc_ftr + "\\cant_auc_21_23.csv" ) 
ftr_05_Z = pd.read_csv( loc_ftr + "\\cant_evt_21_23.csv" )
ftr_06_Z = pd.read_csv( loc_ftr + "\\ref_type_21_23.csv" )  
ftr_07_Z = pd.read_csv( loc_ftr + "\\sdia_auc_21_23.csv" )
ftr_08_Z = pd.read_csv( loc_ftr + "\\srce_auc_21_23.csv" )
ftr_09_Z = pd.read_csv( loc_ftr + "\\rh_encod_21_23.csv" )
ftr_10_Z = pd.read_csv( loc_ftr + "\\main_ahr_21_23.csv" )
ftr_11_Z = pd.read_csv( loc_ftr + "\\kind_evt_21_23.csv" )
ftr_12_Z = pd.read_csv( loc_ftr + "\\frst_auc_21_23.csv" )

In [12]:
Z = Z.merge( ftr_01_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_02_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_03_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_04_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_05_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_06_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_07_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_08_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_09_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_10_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_11_Z, how = "inner", on = "ref_hash" )
Z = Z.merge( ftr_12_Z, how = "inner", on = "ref_hash" )

# <span style="color:orange">  Preparo los datos para entrenar </span>

Con los datos 18-20 + "21-23_sc" entrenamos

In [13]:
Y = st_21_23

In [14]:
X = dt_18_20

### Agregamos los features

In [15]:
ftr_01_X = pd.read_csv( loc_ftr + "\\hora_auc_18_20.csv" ) 
ftr_02_X = pd.read_csv( loc_ftr + "\\cant_clk_18_20.csv" ) 
ftr_03_X = pd.read_csv( loc_ftr + "\\cant_ins_18_20.csv" ) 
ftr_04_X = pd.read_csv( loc_ftr + "\\cant_auc_18_20.csv" ) 
ftr_05_X = pd.read_csv( loc_ftr + "\\cant_evt_18_20.csv" )
ftr_06_X = pd.read_csv( loc_ftr + "\\ref_type_18_20.csv" )  
ftr_07_X = pd.read_csv( loc_ftr + "\\sdia_auc_18_20.csv" )  
ftr_08_X = pd.read_csv( loc_ftr + "\\srce_auc_18_20.csv" )
ftr_09_X = pd.read_csv( loc_ftr + "\\rh_encod_18_20.csv" )
ftr_10_X = pd.read_csv( loc_ftr + "\\main_ahr_18_20.csv" )
ftr_11_X = pd.read_csv( loc_ftr + "\\kind_evt_18_20.csv" )
ftr_12_X = pd.read_csv( loc_ftr + "\\frst_auc_18_20.csv" )

In [16]:
X = X.merge( ftr_01_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_02_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_03_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_04_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_05_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_06_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_07_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_08_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_09_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_10_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_11_X, how = "inner", on = "ref_hash" )
X = X.merge( ftr_12_X, how = "inner", on = "ref_hash" )

## Dropeamos los ref_hash (Quedan los codificados del feature)

In [17]:
X = X.drop( "ref_hash", axis = 1 )
Z = Z.drop( "ref_hash", axis = 1 )

# Prediccion con <span style="color:green"> *CATBOOST*</span> 

### X: Train Data, Y: Train Labels, Z: Test Data, sc_24_26: Test Labels

### Record = iterations=400, learning_rate=0.065, depth=6, RMSE: 94656.147323

In [18]:
from sklearn.model_selection import GridSearchCV

In [19]:
#parameters = {'depth'         : [2,3,4,6,8],
#              'learning_rate' : [0.02, 0.05, 0.06, 0.1, 0.5],
#              'iterations'    : [10, 30, 50, 100, 200]
#                 }

In [20]:
from sklearn.metrics import mean_squared_error, make_scorer
mse = make_scorer( mean_squared_error, greater_is_better = False ) 

In [29]:
model = CatBoostRegressor(loss_function='RMSE')
parameters = {'depth'         : [2,3,6,8],
              'learning_rate' : [0.04, 0.06, 0.08, 0.1],
              'iterations'    : [20, 50, 100, 150]
                 }
grid = GridSearchCV(estimator=model, scoring=mse ,param_grid = parameters, cv = 8, n_jobs=2)
grid.fit(X, Y)    

KeyboardInterrupt: 

In [22]:
    # Results from Grid Search
    print("\n========================================================")
    print(" Results from Grid Search " )
    print("========================================================")    
    
    print("\n The best estimator across ALL searched params:\n",
          grid.best_estimator_)
    
    print("\n The best score across ALL searched params:\n",
          grid.best_score_)
    
    print("\n The best parameters across ALL searched params:\n",
          grid.best_params_)
    
    print("\n ========================================================")


 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 13848725298.219559

 The best parameters across ALL searched params:
 {'depth': 2, 'iterations': 10, 'learning_rate': 0.06}



### Parametros encontrados con grid-searchCV

In [23]:
#Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=10,
                          learning_rate=0.02,
                          depth=2,
                          loss_function='RMSE')

In [24]:
# Fit model
model.fit(X, Y)

0:	learn: 156117.2657169	total: 74.7ms	remaining: 672ms
1:	learn: 154206.4289222	total: 143ms	remaining: 573ms
2:	learn: 152340.1544426	total: 207ms	remaining: 484ms
3:	learn: 150534.7251811	total: 269ms	remaining: 404ms
4:	learn: 148769.2155368	total: 333ms	remaining: 333ms
5:	learn: 147056.7875612	total: 393ms	remaining: 262ms
6:	learn: 145391.9459613	total: 451ms	remaining: 193ms
7:	learn: 143778.0248574	total: 519ms	remaining: 130ms
8:	learn: 142216.2232540	total: 577ms	remaining: 64.1ms
9:	learn: 140690.6620310	total: 634ms	remaining: 0us


In [25]:
preds = model.predict(Z)

In [100]:
pd.DataFrame(preds).head()

,0
0,28568.757555
1,28568.757555
2,15570.757657
3,28568.757555
4,17785.835216


In [101]:
st_24_26.head()

,24_26_st
0,259200.000000
5,259200.000000
7,259200.000000
9,259200.000000
10,250362.048531


### Calculamos el RMSE

In [26]:
from sklearn.metrics import mean_squared_error

In [27]:
rmse = np.sqrt( mean_squared_error(st_24_26, preds) )
print("RMSE: %f" % (rmse) )

RMSE: 141681.450716
